In [22]:
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

from langchain_core.messages import HumanMessage, AIMessage

from utils import print_info

llm = Ollama(model="llama2")

In [14]:
# # combine prompt and llm into simple LLM chain
# prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are world class technical documentation writer."),
#     ("user", "{input}")
# ])

# # add simple output parser to convert the chat message to a string.
# output_parser = StrOutputParser()
# chain = prompt | llm | output_parser
# chain.invoke({"input": "how can langsmith help with testing?"})


In [15]:
# Retrieval Chain
# Retrieval is useful when you have too much data to pass to the LLM directly. 
# You can then use a retriever to fetch only the most relevant pieces and pass those in.
#(1) populate a vector store and use that as a retriever

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()

#(2) index it into a vectorstore. This requires a few components, namely an embedding model and a vectorstore.
embeddings = OllamaEmbeddings()

#(3) use this embedding model to ingest documents into a vectorstore. use a simple local vectorstore, FAISS to build our index
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

#This chain will take an incoming question, look up relevant documents, then pass those documents 
# along with the original question into an LLM and ask it to answer the original question.

#(4)set up the chain that takes a question and the retrieved documents and generates an answer.
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [16]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'\nBased on the provided context, Langsmith can help with testing by allowing users to visualize their test results. This means that Langsmith provides a way to display and analyze the results of tests in a visual format, which can be useful for understanding the outcome of tests and identifying any issues or patterns.'

In [20]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

LangSmith can help with testing by providing a variety of features that make it easier to evaluate and test changes to prompts or chains. Some of the ways LangSmith can help with testing include:

1. Automatic evaluation metrics: LangSmith provides automatic evaluation metrics for getting consistent and scalable information about model behavior. However, human review is still necessary to get the utmost quality and reliability from your application.
2. Annotation queues: LangSmith allows you to manually review and annotate runs through annotation queues. You can select any runs based on criteria like model type or automatic evaluation scores and queue them up for human review.
3. Evaluators: LangSmith provides evaluators that can be specified when initiating a test run, and will evaluate the results once the test run completes. While these evaluators are not perfect, they can guide your eye to examples you should look at and help validate the results of automatic evaluation metrics.
4.

In [23]:
## Conversational Retrieval Chain
# We can still use the create_retrieval_chain function, but we need to change two things:
# The retrieval method should now not just work on the most recent input, but rather should take the whole history into account.
# The final LLM chain should likewise take the whole history into account

# Updating Retrieval
# In order to update retrieval, we will create a new chain. 
# This chain will take in the most recent input (input) and the conversation history (chat_history) 
# and use an LLM to generate a search query.

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content="Skip to main content🦜️🛠️ LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.Over the past two months, we at LangChain have been building and using LangSmith with the goal of bridging this gap. This is our tactical user guide to outline effective ways to use LangSmith and maximize its benefits.On by default\u200bAt LangChain, all of us have LangSmith’s tracing running in the background by default. On the Python side, this is achieved by setting environment variables, which we establish whenever we launch a virtual environment or open our bash shell and leave them set. The same principle app

In [24]:
#  create a new chain to continue the conversation with these retrieved documents in mind.

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [26]:
# test this out end-to-end
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
response = retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})
print(response["answer"])

AI: Of course! LangSmith is a powerful tool for testing and evaluating LLM (Large Language Model) applications. It provides a visualization of the sequence of calls and agents, allowing you to track the exact input to the LLM and see the exact sequence of events. Additionally, it offers a way to collaborate on debugging, collect examples, test & evaluate, and export datasets for further use.

Here are some specific ways LangSmith can help with testing your LLM application:

1. Visualization: LangSmith provides a visual representation of the sequence of calls and agents, making it easier to understand complex chains and agents. This can be especially useful when dealing with non-deterministic sequences, where the exact sequence is difficult to predict ahead of time.
2. Tracking input: LangSmith allows you to track the exact input to the LLM, which can be useful for debugging purposes. By visualizing the input and output of each step in the chain, you can identify potential issues and ma